### Refresher on PSQL and postGres

In [ ]:
## for when I saw struggling to start post gres the old school way
!brew services restart postgresql

In [ ]:
#to open post gres environment
!psql postgres

In [ ]:
#list all databases in postgres local instance
! \l
#Create and drop db
createdb <tablename>
dropdb <tablename>

In [ ]:
Starts psql with a connection to dbname. Optionally use another user than current user
#psql <dbname> [<username>]


In psql:
List all databases on the server, their owners, and user access levels
# \l

Connect to a database named

# \c <dbname>

Show database tables

# \dt

Create table in psql

# create table test123 (col1 int, col2 int, col3 int);commit;

Describe table schema

# \d <tablename>

Quit psql, return to the terminal

# \q

Get help, see list of available commands

# \?

### Data Modelling: setting up

In [1]:
import psycopg2

In [5]:
conn= psycopg2.connect("host= 127.0.0.1 dbname= Udacitydb user=sunshine_e ")

In [7]:
cursor= conn.cursor()

In [8]:
cursor.execute("select * from test123")

In [11]:
cursor.execute("create table check_syntax (info varchar[50],col1 bool, flag char[1] );commit")

In [14]:
cursor.execute("select count(*) from check_syntax")
cursor.fetchall()

[(0,)]

In [23]:
## setting autocommit to True. Making each action a transaction
conn.set_session(autocommit=True)

ProgrammingError: set_session cannot be used inside a transaction

### Data Modelling: Creating tables

In [76]:
conn= psycopg2.connect("host= 127.0.0.1 user=sunshine_e ")
conn.set_session(autocommit=True)

In [77]:
try:
    cursor.execute("select * from udacity.MusicLibrary")
except psycopg2.Error as e:
    print(e)

relation "udacity.musiclibrary" does not exist
LINE 1: select * from udacity.MusicLibrary
                      ^



In [78]:
try:
    cursor.execute("create database Udacity")
except psycopg2.Error as e:
    print(e)

database "udacity" already exists



In [79]:
try:
    conn.close()
except psycopg2.Error as e:
    print(e) 

In [80]:
try:
    conn=psycopg2.connect('host= 127.0.0.1  user=sunshine_e dbname=udacity')
except psycopg2.Error as e:
    print("Error: could not connect to the database")
    print(e)

In [81]:
try:
    cursor= conn.cursor()
except psycopg2.Error as e:
    print("Error: could not connect cursor to the database")
    print(e)

conn.set_session(autocommit=True)

In [82]:
try:
    cursor.execute("CREATE TABLE IF NOT EXISTS music_library(album_name varchar, artist_name varchar, year int);")
except psycopg2.Error as e:
    print("Error: could not create table in the database")
    print(e)

In [83]:
try:
    cursor.execute("select count(*) from music_library")
except psycopg2.Error as e:
    print("Error: could not create table in the database")
    print(e)
    
print(cursor.fetchall())

[(0,)]


In [84]:
try:
    cursor.execute("INSERT INTO music_library (album_name , artist_name , year )\
                   VALUES(%s, %s, %s)", \
                   ('B\'Day', 'Beyonce', 2006))
except psycopg2.Error as e:
    print("Error: could not insert into table in the database")
    print(e)

try:
    cursor.execute("INSERT INTO music_library (album_name , artist_name , year )\
                   VALUES(%s, %s, %s)", \
                   ('Dangerously In love', 'Beyonce', 2003))
except psycopg2.Error as e:
    print("Error: could not insert into table in the database")
    print(e)
    

In [85]:
#postgres allows duplicate values
try:
    cursor.execute("select * from music_library")
except psycopg2.Error as e:
    print("Error: could not select *")
    print(e)

row= cursor.fetchone()
while row:
    print(row)
    row=cursor.fetchone()

("B'Day", 'Beyonce', 2006)
('Dangerously In love', 'Beyonce', 2003)


#### dropping the table and db

In [86]:
try:
    cursor.execute("drop table music_library")
except psycopg2.Error as e:
    print("Error: could not drop table")
    print(e)

#### closing cursor and connection

In [87]:
cursor.close()
conn.close()

### NoSQL data Modelling

In [88]:
import cassandra

ERROR: Could not find a version that satisfies the requirement cassandra (from versions: none)
ERROR: No matching distribution found for cassandra
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
from cassandra.cluster import Cluster
try:
    cluster=Cluster(['127.0.0.1'])
    session= cluster.connect()
except Exception as e:
    print("Error: could notconnect")
    print(e)

In [ ]:
#creating keyspace/DB
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity \
    WITH REPLICATION ={ 'class' : 'SimpleStrategy', 'replication_factor' : 1""")
except Exception as e:
    print(e)

In [ ]:
try:
    session.set_keyspace('udacity'),
except Exception as e:
    print(e)"

In [ ]:
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + \"(year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Apache does not allow for duplicates so thiscan be run multiple times unlike postgres
query = "INSERT INTO music_library (year, artist_name, album_name)"
query = query + " VALUES (%s, %s, %s)"

try:
    session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)

In [ ]:
session.shutdown()
cluster.shutdown()